# [Ateliers: Technologies de l'intelligence Artificielle](https://github.com/wikistat/AI-Frameworks)

<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 
<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" width=400, style="max-width: 150px; display: inline"  alt="Wikistat"/></a>
<a href="http://www.math.univ-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo_imt.jpg" width=400,  style="float:right;  display: inline" alt="IMT"/> </a>
    
</center>

# Traitement Naturel du Langage (NLP) : Catégorisation de Produits Cdiscount

Il s'agit d'une version simplifiée du concours proposé par Cdiscount et paru sur le site [datascience.net](https://www.datascience.net/fr/challenge). Les données d'apprentissage sont accessibles sur demande auprès de Cdiscount mais les solutions de l'échantillon test du concours ne sont pas et ne seront pas rendues publiques. Un échantillon test est donc construit pour l'usage de ce tutoriel.  L'objectif est de prévoir la catégorie d'un produit à partir de son descriptif (*text mining*). Seule la catégorie principale (1er niveau, 47 classes) est prédite au lieu des trois niveaux demandés dans le concours. L'objectif est plutôt de comparer les performances des méthodes et technologies en fonction de la taille de la base d'apprentissage ainsi que d'illustrer sur un exemple complexe le prétraitement de données textuelles. 

Le jeux de données complet (15M produits) permet un test en vrai grandeur du **passage à l'échelle volume** des phases de préparation (*munging*), vectorisation (hashage, TF-IDF) et d'apprentissage en fonction de la technologie utilisée.

La synthèse des résultats obtenus est développée par [Besse et al. 2016](https://hal.archives-ouvertes.fr/hal-01350099) (section 5).

## Partie 2 : Construction des caractéristiques ou *features*, Vectorisation et Word embedding.

Les données textuelles ne peuvent pas être utilisés directement dans les différents algorithmes d'apprentissage statistiques. Nous allons voir dans ce tutoriel plusieurs techniques permettant de traduire les données textuelles sous formes de vecteur numérique : 

Fonction de vectorisation présente dans la librairie `scikit-learn` :

* `One-Hot-Encoder`
* `Tf-Idf`
* `Hashing`

Word Embedding  présente dans la librairie `gensim` :

* `Word2Vec`


## Téléchargement des librairies

In [1]:
#Importation des librairies utilisées
import time
import pandas as pd
import numpy as np
import collections
import itertools
import os
import warnings
warnings.filterwarnings('ignore')


from sklearn.cross_validation import train_test_split

/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Téléchargement des données

On télécharge les données de train et de validation néttoyé et racinisé dans le notebook précédent.

In [2]:
data_valid_clean_stem = pd.read_csv("data/cdiscount_valid_clean_stem.csv").fillna("")
data_train_clean_stem = pd.read_csv("data/cdiscount_train_clean_stem.csv").fillna("")

On créé un dossier dans lequel nous allons sauvegarder les DataFrame constitués des features que l'on va construire dans ce notebook 

In [3]:
DATA_OUTPUT_DIR = "data/features"
if not(os.path.isdir("data/features")):
    os.mkdir("data/features")

Dans un premier temps, en guise d'exemple et pour réduire le temps de calcul, on ne considère que la colonne *Description* de nos `DataFrame` générés dans le calepin précédent.

In [4]:
train_array = data_train_clean_stem["Description"].values
valid_array = data_valid_clean_stem["Description"].values

## Vectorisaton

In [5]:
train_array[0]

'batter original samsung galaxy ace ion mah batter qualit superieur ideal lautonom portabl reduit voir present'

### One-Hot-Encoding

* **One-Hot-Encoding**

L'encodage  *One-Hot-Encoding* des données peut être effectué grâce à la classe `CountVectorizer`de **scikit-learn**

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

extr_cv = CountVectorizer(binary=False)
data_train_OHE = extr_cv.fit_transform(train_array)


**Q** A quoi sert l'argument binary de la classe?

**Q** Sous quel format sont stockées les vecteurs OHE? Pourquoi ce format est-il choisi?

La fonction `get_feature_names` permet d'avoir accès à la liste des mots présents dans l'ensemble des lignes de l'array converti.

In [7]:
vocabulary = extr_cv.get_feature_names()
N_vocabulary = len(vocabulary)
print("Nombre de mots : %d" %N_vocabulary )

Nombre de mots : 56668


**Exercice** Pour la première ligne de votre dataset train. retrouvez l'ensemble des mots constituant cette ligne à partir de l'objet `data_train_OHE`et de `vocabulary` ainsi que le nombre d'occurence de chacun de ces mots dans la ligne.

In [32]:
# %load solution/2_1.py

La même transformation est appliqué sur l'échantillon de validation.

In [23]:
data_valid_OHE = extr_cv.transform(valid_array)
data_valid_OHE

<5000x56668 sparse matrix of type '<class 'numpy.int64'>'
	with 56985 stored elements in Compressed Sparse Row format>

**Q** Que se passe-til pour les mots présents dans le dataset de validation mais qui ne sont pas présent dans le dataset d'apprentissage?

**Q** Pourquoi on ne 're-fit' pas la Classe `CountVectorizer`l'échantillon de validation. 

### TF-IDF¶

* **TF-IDF**. Le TF-IDF permet de faire ressortir l'importance relative de chaque mot $m$ (ou couples de mots consécutifs) dans un texte-produit ou un descriptif $d$, par rapport à la liste entière des produits. La fonction $TF(m,d)$ compte le nombre d'occurences du mot $m$ dans le descriptif $d$. La fonction $IDF(m)$ mesure l'importance du terme dans l'ensemble des documents ou descriptifs en donnant plus de poids aux termes les moins fréquents car considérés comme les plus discriminants (motivation analogue à celle de la métrique du chi2 en anamlyse des correspondance). $IDF(m,l)=\log\frac{D}{f(m)}$ où $D$ est le nombre de documents, la taille de l'échantillon d'apprentissage, et $f(m)$ le nombre de documents ou descriptifs contenant le mot $m$. La nouvelle variable ou *features* est $V_m(l)=TF(m,l)\times IDF(m,l)$.

* Comme pour les transformations des variables quantitatives (centrage, réduction), la même transformation c'est-à-dire les mêmes pondérations, est calculée sur l'achantillon d'apprentissage et appliquée à celui de test.

On utiliser la fonction `TfidfVectorizer` qui permet de parser également le texte

Dans un premier temps, on fixe le paramètre `norm` = False afin de rendre les résultats plus explicite et analyser les sorties 

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer(ngram_range=(1,1), norm = False)
data_train_TFIDF = vec.fit_transform(train_array)

**Q** A quoi sert l'argument ngram_range?

Constatez que `data_train_TFIDF`est stocké sous le même format que `data_train_OHE` et que la taile du vocabulaire est la même.

In [28]:
vocabulary = vec.get_feature_names()
N_vocabulary = len(vocabulary)
N_vocabulary

56668

**Exercice** Pour la première ligne de votre dataset train. retrouvez l'ensemble des mots constituant cette ligne à partir de l'objet `data_train_TFIDF`et de `vocabulary` ainsi que la valeur de l'idf, du tf et du poids tfidf de chacun de ces mots dans la ligne

In [46]:
# %load solution/2_2.py

**Q** Commentez les valeurs de l'idf pour chacun des mots.

**Q** Comment evolue les poids en changeant les paramètre *smooth idf* et *sublinear_tf* de la méthode `TfidfVectorizer`?

**Exercice** Changez l'argument *ngram_range* de la méthode `TfidfVectorizer` et re-affichez les résultats.

On applique maintenant le `vectorizer` sur le jeu de données de validation

In [34]:
data_valid_TFIDF = vec.transform(valid_array)
data_valid_TFIDF

<5000x56668 sparse matrix of type '<class 'numpy.float64'>'
	with 56985 stored elements in Compressed Sparse Row format>

**ATENTION** Si le tf est recalculé pour chaque ligne, le même idf est utilisé

In [51]:
# %load solution/2_2bis.py

### Hashing

Le **Hashage**. Il permet de réduire l'espace des variables (taille du dictionnaire) en un nombre limité et fixé a priori `n_hash` de caractéristiques. Il repose sur la définition d'une fonction de hashage, $h$ qui à un indice $j$ défini dans l'espace des entiers naturels, renvoie un indice $i=h(j)$ dans dans l'espace réduit (1 à n_hash) des caractéristiques. Ainsi le poids de l'indice $i$, du nouvel espace, est l'association de tous les poids d'indice $j$ tels que $i=h(j)$ de l'espace originale. Ici, les poids sont associés d'après la méthode décrite par Weinberger et al. (2009).

N.B. $h$ n'est pas généré aléatoirement. Ainsi pour un même fichier d'apprentissage (ou de test) et pour un même entier n_hash, le résultat de la fonction de hashage est identique

Au contraire des classe `CountVectorizer`et `TfidfVectorizer`, la classe `FeatureHasher` prend en entré un dictionnaire d'occurence des mots.

In [53]:
train_dict_array  = list(map(lambda x : collections.Counter(x.split(" ")), train_array))
train_dict_array[0]

Counter({'ace': 1,
         'batter': 2,
         'galaxy': 1,
         'ideal': 1,
         'ion': 1,
         'lautonom': 1,
         'mah': 1,
         'original': 1,
         'portabl': 1,
         'present': 1,
         'qualit': 1,
         'reduit': 1,
         'samsung': 1,
         'superieur': 1,
         'voir': 1})

In [54]:
from sklearn.feature_extraction import FeatureHasher
nb_hash = 300
feathash = FeatureHasher(nb_hash)
data_train_hash = feathash.fit_transform(train_dict_array)

Constatez que `data_train_hash` est stocké sous le même format que `data_train_OHE` ou `data_train_TFIDF`.  

**Q** Que dire cependant de sa dimension?

La cellule suivante permet d'afficher le poids de chacun des indices dans le nouvel espace.

In [60]:
ir = 0
rw = data_train_hash.getrow(ir)
print("Liste des tokens racinisé de la première ligne : " + train_array[0])
pd.DataFrame([(v, k)  for k,v in zip(rw.data,rw.indices)], columns=["indices","weight"])

Liste des tokens racinisé de la première ligne : batter original samsung galaxy ace ion mah batter qualit superieur ideal lautonom portabl reduit voir present


,indices,weight
0,3,1.0
1,4,1.0
2,14,1.0
3,21,-1.0
4,30,1.0
5,74,1.0
6,113,-1.0
7,131,1.0
8,173,1.0
9,175,1.0


**Q** Que pouvez-vous dire des poids?

La taille de la matrice a donc été très réduit par rapport au vectorizer, TFIDF ou One-Hot-Enconding. Cependant, il n'y a pas de fonction inverse transform ce qui peut rendre le résultat moin compréhensible.


Il est possible de combiner le `FeatureHasher`avec un autre vectorizer comme le TFIDF. 

C'est cette fois la classe `TFIDFTransformer` qui est utilisé. Celle ci ne ne considère pas des string en entré mais l'array en sortie de l'étape de Hashage. Les mots sont les `nb_hash`indices sélectionnés et le tf pour chaque individu sont les poids calculé par la fonction de hasage.

In [61]:
from sklearn.feature_extraction.text import TfidfTransformer

vec =  TfidfTransformer(norm = False)
data_train_HTfidf = vec.fit_transform(data_train_hash)
data_train_HTfidf

<95000x300 sparse matrix of type '<class 'numpy.float64'>'
	with 1082962 stored elements in Compressed Sparse Row format>

In [63]:
ir = 0

rw = data_train_HTfidf.getrow(ir)
print(train_array[ir])
pd.DataFrame([(ind, vec.idf_[ind], w/vec.idf_[ind], w)  for w,ind in zip(rw.data, rw.indices)], columns=["indices","idf","tf","weight"])


batter original samsung galaxy ace ion mah batter qualit superieur ideal lautonom portabl reduit voir present


,indices,idf,tf,weight
0,288,4.510436,-1.0,-4.510436
1,274,1.310477,1.0,1.310477
2,272,4.510788,-1.0,-4.510788
3,220,1.326674,-1.0,-1.326674
4,204,3.179447,2.0,6.358894
5,175,3.764130,1.0,3.764130
6,173,3.332079,1.0,3.332079
7,131,3.910628,1.0,3.910628
8,113,5.015058,-1.0,-5.015058
9,74,3.637848,1.0,3.637848


###  Build and Save Vectorize Vector

Afin de comparer l'effet de ces différentes vectorisation sur l'apprentissage, nous allons sauvegarder ces dernières sur les machines.

De nombreux paramètres sont à régler ce qui entraine donc un un très grand nombre de combinaison.

ici nous allons créer 4 jeu de données avec `CountVectorizer` et `TFIDF` chacun avec et sans hashage de taille 300.

Il est evidemment possible de tester d'autre combinaison. Libre à vous de les tester ;) 

### Fonction de Vectorisation

on créé deux fonctions `vectorizer_train` and `apply_vectorizer` afin de générer automatiquement différent dataframe d'apprentissage et de validation vectorisé.

In [64]:
def vectorizer_train(df, columns=['Description'], nb_hash=None, nb_gram = 1, vectorizer = "tfidf" , binary = False):
    
    data_array = [" ".join(line) for line in df[columns].values]
    
    # Hashage
    if nb_hash is None:
        feathash = None
        if vectorizer == "tfidf":
            vec = TfidfVectorizer(ngram_range=(1,nb_gram))
            data_vec = vec.fit_transform(data_array)
        else:
            vec = CountVectorizer(binary=binary)
            data_vec = vec.fit_transform(data_array)
    else:
        data_dic_array = [collections.Counter(line.split(" ")) for line in data_array]
        feathash = FeatureHasher(nb_hash)
        data_hash = feathash.fit_transform(data_dic_array)
        
        if vectorizer=="tfidf":
            vec =  TfidfTransformer()
            data_vec =  vec.fit_transform(data_hash)
        else:
            vec = None
            data_vec = data_hash

    return vec, feathash, data_vec



def apply_vectorizer(df, vec, feathash, columns =['Description', 'Libelle', 'Marque']):
    
    data_array = [" ".join(line) for line in df[columns].values]
    
    #Hashage
    if feathash is None:
        data_hash = data_array
    else:
        data_dic_array = [collections.Counter(line.split(" ")) for line in data_array]
        data_hash = feathash.transform(data_dic_array)
    
    if vec is None:
        data_vec = data_hash
    else:
        data_vec = vec.transform(data_hash)
    return data_vec


In [65]:
parameters = [[None, "count"],
              [300, "count"],
              [None, "tfidf"],
              [300, "tfidf"]]

from scipy import sparse

for nb_hash, vectorizer in parameters:
    ts = time.time()
    vec, feathash, data_train_vec = vectorizer_train(data_train_clean_stem, nb_hash=nb_hash, vectorizer = vectorizer)
    data_valid_vec = apply_vectorizer(data_valid_clean_stem, vec, feathash)
    te = time.time()
    
    print("nb_hash : " + str(nb_hash) + ", vectorizer : " + str(vectorizer))
    print("Runing time for vectorization : %.1f seconds" %(te-ts))
    print("Train shape : " + str(data_train_vec.shape))
    print("Valid shape : " + str(data_valid_vec.shape))

    
    sparse.save_npz(DATA_OUTPUT_DIR +"/vec_train_nb_hash_" + str(nb_hash) + "_vectorizer_" + str(vectorizer), data_train_vec)
    sparse.save_npz(DATA_OUTPUT_DIR +"/vec_valid_nb_hash_" + str(nb_hash) + "_vectorizer_" + str(vectorizer), data_valid_vec)
    

nb_hash : None, vectorizer : count
Runing time for vectorization : 2.2 seconds
Train shape : (95000, 56668)
Valid shape : (5000, 56668)
nb_hash : 300, vectorizer : count
Runing time for vectorization : 1.8 seconds
Train shape : (95000, 300)
Valid shape : (5000, 300)
nb_hash : None, vectorizer : tfidf
Runing time for vectorization : 2.0 seconds
Train shape : (95000, 56668)
Valid shape : (5000, 56668)
nb_hash : 300, vectorizer : tfidf
Runing time for vectorization : 1.7 seconds
Train shape : (95000, 300)
Valid shape : (5000, 300)


## Word2Vec

Dans cette partie, des modèles `Word2Vec`Seront créés à l'aide de la librairie [**gensim**](https://radimrehurek.com/gensim/index.html). 

In [74]:
import gensim

### Build Word2Vec model

La fonction `gensim.models.Word2Vec`qui permet de construire des modèle Word2Vec prend en entrée une liste de tokens. On tranformer donc nos données dans un premier temps

In [75]:
train_array_token = [line.split(" ") for line in train_array]
valid_array_token = [line.split(" ") for line in valid_array]
train_array_token[0]

['batter',
 'original',
 'samsung',
 'galaxy',
 'ace',
 'ion',
 'mah',
 'batter',
 'qualit',
 'superieur',
 'ideal',
 'lautonom',
 'portabl',
 'reduit',
 'voir',
 'present']

Cette fonction contient un grand nombre d' [arguments](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec). Le but de ce TP n'est pas d'optimiser les paramètres de ce modèle mais de le comprendre. Nous allons donc fixer quelques argument par défault : 

* Features_dimension = 300 : Dimension de l'espace des features (d'*embedding*) qui sera crée.
* hs = 0
* negative = 10

**Q** A quoi servent les arguments *hs* et *negative*? Quels influences ces arguments ont sur le modèle avec les valeurs défini précédemment?

In [68]:
Features_dimension = 300
hs = 0
negative = 10

Nous allons créer deux modèles :

* Un modèle **skip-sgram**, sg = 1
* Un modèle **CBOW**, sg = 0

In [82]:
sg = 1
print("Start learning skip-gram Word2Vec")
ts = time.time()
model_sg = gensim.models.Word2Vec(X, sg=sg, hs=hs, negative=negative, min_count=1, size=Features_dimension)
te = time.time()
t_learning = te-ts
print("Learning time : %.2f Word2Vec" %t_learning)


sg = 0
print("Start learning CBOW Word2Vec")
ts = time.time()
model_cbow = gensim.models.Word2Vec(X, sg=sg, hs=hs, negative=negative, min_count=1, size=Features_dimension)
te = time.time()
t_learning = te-ts
print("Learning time : %.2f Word2Vec" %t_learning)




Start learning skip-gram Word2Vec
Learning time : 36.95 Word2Vec
Start learning CBOW Word2Vec
Learning time : 11.05 Word2Vec


**Q** Que dire du temps d'apprentissage de ces deux modèles? D'ou vient cette différence?

### Pre-Trained Model

Comme pour les réseaux de convolution, des modèles pré-entrainés de Word2Vec éxistent également. 
Le plus célèbre et le plus utilisé étant [`GoogleNewsVectors`](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit) appris sur plus de 100 milliard de mots à partir des articles de GoogleNews. Cependant ce modèle est en anglais, et ne nous saura pas utile ici.

On utilisera des modèle appris dans le projet suivant [https://github.com/Kyubyong/wordvectors](https://github.com/Kyubyong/wordvectors) appris sur 1Giga d'articles de wikipedia. 

Vous pouvez télécharger ce modèle en suivant ce [lien](https://we.tl/t-QMWo949gSf). Dezipez-le puis téléchargez le modèle en indiquant la direction du fichier "fr/bin"

In [83]:
model_online_dir = "data/fr/fr.bin"
#model_online_dir = "ACOMPLETER/fr.bin"
model_online = gensim.models.Word2Vec.load(model_online_dir)


### Most similar world

Nous allons maintenant comparérer quelques propriétés de chacun de ces modèles.

In [ ]:
import nltk 
stemmer=nltk.stem.SnowballStemmer('french')

In [70]:
model.predict_output_word(["homm"])

In [71]:
term="homme"

df_ = []
columns = []
for model_name, model in model_dic.items():
    token = stemmer.stem(term) if "online"!=model_name else term
    mpow = model.wv.most_similar([token])
    if mpow is None:
        df_.append(["" for k in range(10)])
    else:
        df_.append([k[0] for k in mpow])
    columns.append(model_name)
print("Most similar words for word : "+term)
pd.DataFrame(np.array(df_).T, columns=columns)

Most similar words for word : homme


,CBOW,skip-gram,online
0,femm,unisex,aventurier
1,unisex,chronograph,aristocrate
2,arman,arman,garçon
3,bermud,umbro,vieillard
4,diesel,army,délinquant
5,emporio,emporio,écrivain
6,norway,ralph,politicien
7,womag,chrono,escroc
8,navy,lauren,orateur
9,geographical,calvin,érudit


**Exercice** Affichez les résultats pour d'autres mots.

In [72]:
terms_positif = ["femme","roi"]
terms_negatif = ["homme"]

df_ = []
columns = []
for model_name, model in model_dic.items():
    token_positif = [stemmer.stem(term) if "online"!=model_name else term for term in terms_positif]
    token_negativ = [stemmer.stem(term) if "online"!=model_name else term for term in terms_negatif]
    
    mpow = model.wv.most_similar(positive=token_positif, negative=token_negativ)
    if mpow is None:
        df_.append(["" for k in range(10)])
    else:
        df_.append([k[0] for k in mpow])
    columns.append(model_name)
pd.DataFrame(np.array(df_).T, columns=columns)

,CBOW,skip-gram,online
0,romant,jungl,reine
1,rev,savan,belle-mère
2,lapin,marionnet,brunehilde
3,jungl,joyeux,régente
4,eleph,felin,reine-mère
5,runeun,paon,belle-sœur
6,coquelicot,lilipinso,veuve
7,amour,ruin,mère
8,halloween,tendress,demi-sœur
9,calin,capelin,nièce


In [73]:
terms_positif = ["espagne","paris"]
terms_negatif = ["france"]

df_ = []
columns = []
for model_name, model in model_dic.items():
    token_positif = [stemmer.stem(term) if "online"!=model_name else term for term in terms_positif]
    token_negativ = [stemmer.stem(term) if "online"!=model_name else term for term in terms_negatif]
    
    mpow = model.wv.most_similar(positive=token_positif, negative=token_negativ)
    if mpow is None:
        df_.append(["" for k in range(10)])
    else:
        df_.append([k[0] for k in mpow])
    columns.append(model_name)
pd.DataFrame(np.array(df_).T, columns=columns)

,CBOW,skip-gram,online
0,sweetness,dublin,ségovie
1,flex,arizon,madrid
2,flocon,zurich,alicante
3,etire,autrich,aix-en-provence
4,opta,copenhagen,ancône
5,capitonnag,buffalo,séville
6,canc,janeiro,malaga
7,liberty,foxy,andalousie
8,cercl,atlant,alger
9,sophistique,toronto,avignon


### Predict output word

In [ ]:
terms = ["voir","la"]


df_ = []
columns = []
for model_name, model in model_dic.items():
    tokens = [stemmer.stem(term) if "online"!=model_name else term for term in terms]
    
    mpow = model.predict_output_word(tokens)
    if mpow is None:
        df_.append(["" for k in range(10)])
    else:
        df_.append([k[0] for k in mpow])
    columns.append(model_name)
pd.DataFrame(np.array(df_).T, columns=columns)

In [ ]:
terms = ["coque",'pour',"samsung"]


df_ = []
columns = []
for model_name, model in model_dic.items():
    tokens = [stemmer.stem(term) if "online"!=model_name else term for term in terms]
    
    mpow = model.predict_output_word(tokens)
    if mpow is None:
        df_.append(["" for k in range(10)])
    else:
        df_.append([k[0] for k in mpow])
    columns.append(model_name)
pd.DataFrame(np.array(df_).T, columns=columns)

In [ ]:
terms = ["homme"]


df_ = []
columns = []
for model_name, model in model_dic.items():
    tokens = [stemmer.stem(term) if "online"!=model_name else term for term in terms]
    
    mpow = model.predict_output_word(tokens)
    if mpow is None:
        df_.append(["" for k in range(10)])
    else:
        df_.append([k[0] for k in mpow])
    columns.append(model_name)
pd.DataFrame(np.array(df_).T, columns=columns)

In [ ]:
terms = ["femme"]


df_ = []
columns = []
for model_name, model in model_dic.items():
    tokens = [stemmer.stem(term) if "online"!=model_name else term for term in terms]
    
    mpow = model.predict_output_word(tokens)
    if mpow is None:
        df_.append(["" for k in range(10)])
    else:
        df_.append([k[0] for k in mpow])
    columns.append(model_name)
pd.DataFrame(np.array(df_).T, columns=columns)

### Build Features

#### Our Build Model

In [ ]:
data_valid_clean = pd.read_csv("data/cdiscount_valid_clean.csv").fillna("")
data_train_clean = pd.read_csv("data/cdiscount_train_clean.csv").fillna("")

train_array_token_wstem = [line.split(" ") for line in data_train_clean["Description"].values]
valid_array_token_wstem = [line.split(" ") for line in data_valid_clean["Description"].values]

In [ ]:
def get_features_mean(lines):
    features = [MODEL[x] for x  in lines if x in MODEL]
    if features == []:   
        fm =np.ones(F_SIZE)
    else :
        fm = np.mean(features,axis=0)
    return fm

def get_matrix_features_means(X):
    X_embedded_ = list(map(get_features_mean, X))
    X_embedded = np.vstack(X_embedded_)
    return X_embedded



In [ ]:
for model_name in ["CBOW","skip-gram", "online"]:
    
    if "online" == model_name:
        X_train = train_array_token_wstem
        X_valid = valid_array_token_wstem
    else:
        X_train = train_array_token
        X_valid = valid_array_token
    
    model = model_dic[model_name]
    MODEL = model
    F_SIZE = Features_dimension

    ts = time.time()
    X_embedded_train = get_matrix_features_means(X_train)
    te = time.time()
    t_train = te-ts
    #np.save(embedded_train_dir, X_embedded_train)
    print("Time conversion : %d seconds"%t_train)
    print("Shape Matrix : (%d,%d)"%X_embedded_train.shape)
    np.save(DATA_OUTPUT_DIR +"/embedded_train_nb_hash_"+model_name, X_embedded_train)

    
    ts = time.time()
    X_embedded_valid = get_matrix_features_means(X_valid)
    te = time.time()
    t_valid = te-ts
    #np.save(embedded_valid_dir, X_embedded_valid)
    print("Time conversion : %d seconds"%t_valid)
    print("Shape Matrix : (%d,%d)"%X_embedded_valid.shape)
    np.save(DATA_OUTPUT_DIR +"/embedded_valid_nb_hash_"+model_name, X_embedded_valid)
    
    metadata = {"t_train" : t_train, "t_valid" : t_valid, "sg":sg}
    print(metadata)

In [ ]:
model["homme"]

In [ ]:
model.wv.syn0[model.wv.vocab["homme"].index] == model["homme"]

In [ ]:
model.syn1neg.shape

In [ ]:
vh = model.wv.vocab["homme"]
vh.index